<a href="https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/object_detection_for_image_cropping/aves/aves_generate_crops_tf2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Faster-RCNN and SSD in Tensorflow to automatically crop images of birds 
---   
*Last Updated 7 May 2021*.  
-Runs in Python 3 with Tensorflow 2.0-   
Using [Faster-RCNN](https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1) and [SSD](https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2) models pretrained on [MS COCO 2017](https://cocodataset.org/#explore) as methods to do customized, large-scale image processing with Tensorflow. Using the location and dimensions of the detected birds, images will be cropped to square dimensions that are centered and padded around the object(s) of interest (ie birds). Pre-trained models are used for "out of the box" inference on images of birds of varying dimensions and resolutions.

Code is modified from [here](https://medium.com/@nickbortolotti/tensorflow-object-detection-api-in-5-clicks-from-colaboratory-843b19a1edf1). The [Tensorflow Object Detection API Tutorial](https://github.com/tensorflow/models/tree/master/research/object_detection) was also used as a reference. The [Tensorflow Object Detection API](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html#tensorflow-models-installation) is used for building custom models for object detection. 

## Installs & Imports
---

In [ ]:
# Mount google drive to export image cropping coordinate file(s)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# For running inference on the TF-Hub module
import tensorflow as tf
import tensorflow_hub as hub

# For downloading image
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# For drawing onto images
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# For measuring the inference time
import time

# For working with data
import numpy as np
import pandas as pd
import os
import csv
import urllib

# Print Tensorflow version
print('Tensorflow Version: %s' % tf.__version__)

# Check available GPU devices
print('The following GPU devices are available: %s' % tf.test.gpu_device_name())

# Define functions

# Read in data file exported from "Combine output files A-D" block above
def read_datafile(fpath, sep="\t", header=0, disp_head=True):
    """
    Defaults to tab-separated data files with header in row 0
    """
    try:
        df = pd.read_csv(fpath, sep=sep, header=header)
        if disp_head:
          print("Data header: \n", df.head())
    except FileNotFoundError as e:
        raise Exception("File not found: Enter the path to your file in form field and re-run").with_traceback(e.__traceback__)
    
    return df

# To load image in and do something with it
def load_img(path): 
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  return img

# To display loaded image
def display_image(image):
  fig = plt.figure(figsize=(20, 15))
  plt.grid(False)
  plt.imshow(image)

# For reading in images from URL and passing through TF models for inference
def download_and_resize_image(url, new_width=256, new_height=256, #From URL
                              display=False):
  _, filename = tempfile.mkstemp(suffix=".jpg")
  response = urlopen(url)
  image_data = response.read()
  image_data = BytesIO(image_data)
  pil_image = Image.open(image_data)
  im_h, im_w = pil_image.size
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
  pil_image_rgb = pil_image.convert("RGB")
  pil_image_rgb.save(filename, format="JPEG", quality=90)
  #print("Image downloaded to %s." % filename)
  if display:
    display_image(pil_image)
  return filename, im_h, im_w

## Generate cropping coordinates for images
---
Run EOL 20k image bundles through pre-trained object detection models and save results in 4 batches (A-D). 

#### Set up model parameters

In [ ]:
import csv
import os

# Load Pre-trained model from Tensorflow Hub
# TO DO: Choose model to run inference with (both trained on MS COCO 2017)
model = "SSD MobileNet v2" #@param ["SSD MobileNet v2", "Faster RCNN Resnet 50"]
if 'SSD' in model: 
  module_handle = "https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2"
  mod_abbv = '_ssd'
elif 'RCNN' in model:
  module_handle = "https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1"
  mod_abbv = '_rcnn'
else:
  print('Error: Model name does not correspond to module_handle')
print('Loading {} from TF-Hub...'.format(model))
detector = hub.load(module_handle)

# MS COCO 2017 Label Map (# output by models corresp. to text lables)
# Note: You can modify "filter" to choose detection results for any of these categories
filter = "bird" #@param {type:"string"}
label_map = {1: 'person', 2: 'bicycle', 3: 'car', 4: 'motorcycle', 5: 'airplane',
              6: 'bus', 7: 'train', 8: 'truck', 9: 'boat', 10: 'traffic light',
              11: 'fire hydrant', 13: 'stop sign', 14: 'parking meter', 15: 'bench',
              16: 'bird', 17: 'cat', 18: 'dog', 19: 'horse', 20: 'sheep', 21: 'cow',
              22: 'elephant', 23: 'bear', 24: 'zebra', 25: 'giraffe', 27: 'backpack',
              28: 'umbrella', 31: 'handbag', 32: 'tie', 33: 'suitcase', 34: 'frisbee',
              35: 'skis', 36: 'snowboard', 37: 'sports ball', 38: 'kite',
              39: 'baseball bat', 40: 'baseball glove', 41: 'skateboard', 42: 'surfboard',
              43: 'tennis racket', 44: 'bottle', 46: 'wine glass', 47: 'cup', 48: 'fork',
              49: 'knife', 50: 'spoon', 51: 'bowl', 52: 'banana', 53: 'apple',
              54: 'sandwich', 55: 'orange', 56: 'broccoli', 57: 'carrot', 58: 'hot dog',
              59: 'pizza', 60: 'donut', 61: 'cake', 62: 'chair', 63: 'couch',
              64: 'potted plant', 65: 'bed', 67: 'dining table', 70: 'toilet',
              72: 'tv', 73: 'laptop', 74: 'mouse', 75: 'remote', 76: 'keyboard',
              77: 'cell phone', 78: 'microwave', 79: 'oven', 80: 'toaster', 81: 'sink',
              82: 'refrigerator', 84: 'book', 85: 'clock', 86: 'vase', 87: 'scissors',
              88: 'teddy bear', 89: 'hair drier', 90: 'toothbrush'}

# For handling bounding boxes
def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color,
                               font,
                               thickness=4,
                               display_str_list=()):
  """Adds a bounding box to an image."""
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
  draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

  # Adjust display string placement if out of bounds
  display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
  # Each display_str has a top and bottom margin of 0.05x.
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)
  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = top + total_display_str_height
  # Reverse list and print from bottom to top.
  for display_str in display_str_list[::-1]:
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05 * text_height)
    draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                    (left + text_width, text_bottom)],
                   fill=color)
    draw.text((left + margin, text_bottom - text_height - margin),
              display_str,
              fill="black",
              font=font)
    text_bottom -= text_height - 2 * margin

# TO DO: Set the maximum number of detections to keep per image
max_boxes = 10 #@param {type:"slider", min:0, max:100, step:10}

def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.1):
  """Overlay labeled boxes on an image with formatted scores and label names."""
  if max_boxes:
    max_boxes = max_boxes
  colors = list(ImageColor.colormap.values())

  try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                              25)
  except IOError:
    print("Font not found, using default font.")
    font = ImageFont.load_default()

  for i in range(0, max_boxes):
    if scores[0][i] >= min_score:
      ymin, xmin, ymax, xmax = tuple(boxes[0][i])
      display_str = "{}: {}%".format(label_map[class_names[0][i]],
                                     int(100 * scores[0][i]))
      color = colors[hash(class_names[0][i]) % len(colors)]
      image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
      if filter in display_str: # Only the filtered class is shown on images
        draw_bounding_box_on_image(
          image_pil,
          ymin,
          xmin,
          ymax,
          xmax,
          color,
          font,
          display_str_list=[display_str])
      np.copyto(image, np.array(image_pil))
  return image

# Define start and stop indices in EOL bundle for running inference   
def set_start_stop():
    # To test with a tiny subset, use first 5 bundle images
    if test_with_tiny_subset:
        start=0
        stop=5
    # To run inference on 4 batches of 5k images each
    elif "_a." in outfpath: # batch a is from 0-5000
        start=0
        stop=5000
    elif "_b." in outfpath: # batch b is from 5000-1000
        start=5000
        stop=10000
    elif "_c." in outfpath: # batch c is from 10000-15000
        start=10000
        stop=15000
    elif "_d." in outfpath: # batch d is from 15000-20000
        start=15000
        stop=20000
    
    return start, stop

# For running inference
def run_detector(image_url):
  image_path, im_h, im_w = download_and_resize_image(image_url, 640, 480)
  img = load_img(image_path)

  converted_img  = tf.image.convert_image_dtype(img, tf.uint8)[tf.newaxis, ...]
  start_time = time.time()
  result = detector(converted_img)
  end_time = time.time()

  result = {key:value.numpy() for key,value in result.items()}

  print("Found %d objects." % result["num_detections"])
  print("Inference time: %s" % format(end_time-start_time, '.2f'))

  image_with_boxes = draw_boxes(img.numpy(), result["detection_boxes"],
      result["detection_classes"], result["detection_scores"])

  # Export bounding boxes to file in Google Drive
  with open(outfpath, 'a') as out_file:
            tsv_writer = csv.writer(out_file, delimiter='\t')
            img_id = os.path.splitext((os.path.basename(image_url)))[0]
            # Write one row per detected object with bounding box coordinates
            num_detections = min(int(result["num_detections"][0]), max_boxes)
            for i in range(0, num_detections):
                class_id = str(label_map[result["detection_classes"][0][i]])
                if filter in class_id: # Only writes rows for filtered class
                      ymin = result["detection_boxes"][0][i][0]
                      xmin = result["detection_boxes"][0][i][1]
                      ymax = result["detection_boxes"][0][i][2]
                      xmax = result["detection_boxes"][0][i][3]
                      tsv_writer.writerow([img_id, class_id, 
                          xmin, ymin, xmax, ymax, im_w, im_h, image_url])
  return image_with_boxes

print('Model loaded and functions defined! \nGo to next steps for running inference on images.')

#### Test: Run inference on a couple images from URLs

In [ ]:
# TO DO: Type in image URLs 1-3 using form fields to right
url_1 = "https://content.eol.org/data/media/7e/9c/7a/542.15445377044.jpg" #@param {type:"string"}
url_2 = "https://content.eol.org/data/media/81/1c/0d/542.7816025222.jpg" #@param {type:"string"}
url_3 = "https://content.eol.org/data/media/7e/3c/0b/542.10578857864.jpg" #@param {type:"string"}
image_urls = [url_1, url_2, url_3]

# Display detection results on images
display_results = True

# Set temporary outfile for tagging results
outfpath = "temp_outfile.tsv"

# Loop through EOL image bundle to add bounding boxes to images
print("Running inference on images")
for im_num, image_url in enumerate(image_urls, start=1):
  try:
    image_wboxes = run_detector(image_url)
    if display_results:
      display_image(image_wboxes)
    # Display progress message after each image
    print('Inference complete for image {} of {}'.format(im_num, len(image_urls)))

  except:
    print('Check if URL from {} is valid'.format(image_url))
  
  os.remove(outfpath) # Delete temporary outfile

### Generate crops: Run inference on EOL images & save results for cropping
Use 20K EOL Aves image bundle to get bounding boxes of detected birds. Results are saved to [crops_file].tsv. Running in 4 batches of 5K images is recommended in case of Colab timeouts.

In [ ]:
# So URL's don't get truncated in display
pd.set_option('display.max_colwidth',1000)

# Read in EOL image bundle dataframe
# TO DO: Type in image bundle address using form field to right
bundle = "https://editors.eol.org/other_files/bundle_images/files/images_for_Aves_20K_breakdown_download_000001.txt" #@param {type:"string"}
df = read_datafile(bundle, sep='\n', header=None)
df.columns = ['url']
print('\n EOL image bundle head:\n{}'.format(df.head()))

# Write header row of output tagging file
# TO DO: Change file name for each bundle/run
# Note: If running in 4 batches of 5k images per 20k image bundle (reccomended), use a/b/c/d for each batch
base = '/content/drive/My Drive/train/results/'
crops_file = "aves_cropcoords_tf2_a" #@param ["aves_cropcoords_tf2_a", "aves_cropcoords_tf2_b", "aves_cropcoords_tf2_c", "aves_cropcoords_tf2_d"] {allow-input: true}
outfpath = base + crops_file.rsplit('_',1)[0] + mod_abbv + '_' + crops_file.rsplit('_',1)[1] + '.tsv'
print('\n Cropping file will be saved to:\n{}'.format(outfpath))

# Write header row of output tag file
with open(outfpath, 'a') as out_file:
                  tsv_writer = csv.writer(out_file, delimiter='\t')
                  tsv_writer.writerow(["img_id", "class_id", 
                            "xmin", "ymin", "xmax", "ymax", "im_width", "im_height", "url"])

In [ ]:
# Run inference on images

# Test with a smaller subset than 5k images?
# TO DO: If yes, check test_with_tiny_subset box
test_with_tiny_subset = True #@param {type: "boolean"}

# Display detection results on images?
# TO DO: Check display_results box if "Yes"
# Note: Only run for <50 images at a time
display_results = False #@param {type:"boolean"}

# Loop through EOL image bundle to add bounding boxes to images
print("Running inference on images")
start, stop = set_start_stop()
for i, row in df.iloc[start:stop].iterrows():
  try:
    image_wboxes = run_detector(df['url'][i])
    if display_results:
      display_image(image_wboxes)
    
    # Display progress message after each image
    print('Inference complete for image {} of {}'.format(i+1, (stop-start)))

  except:
    print('Check if URL from {} is valid'.format(df['url'][i]))

## Post-process detection results
--- 
Combine output files for batches A-D. Then, convert detection boxes into square, centered thumbnail cropping coordinates.

#### Merge batch output files A-D

In [ ]:
# So URL's don't get truncated in display
pd.set_option('display.max_colwidth',1000)
pd.options.display.max_columns = None

# Get name of ONE output file (any of A-D)
# TO DO: If you just ran "Generate crops" above, you do not need to enter anything
# TO DO: If you ran "Generate crops" during a previous session, enter the path for ONE output file
if 'outfpath' not in locals() or globals():
  outfpath = "/content/drive/My Drive/train/results/aves_cropcoords_tf2_ssd_d.tsv" #@param {type:"string"}

# Combine 4 batches of detection box coordinates to one dataframe
base =  os.path.splitext(outfpath)[0].rsplit('_',1)[0] + '_'
exts = ['a.tsv', 'b.tsv', 'c.tsv', 'd.tsv']
all_filenames = [base + e for e in exts]
df = pd.concat([pd.read_csv(f, sep='\t', header=0, na_filter = False) for f in all_filenames], ignore_index=True)

# Write results to tsv
print("New concatenated dataframe with all 4 batches: \n", df.head())
concat_outfpath = base + 'concat.tsv'
df.to_csv(concat_outfpath, sep='\t', index=False)

#### Combine individual detection boxes into one "superbox" per image

In [ ]:
# Define functions

from functools import reduce
from urllib.error import HTTPError
# So URL's don't get truncated in display
pd.set_option('display.max_colwidth',1000)
pd.options.display.max_columns = None

# Convert normalized detection coordinates (scaled to 0,1) to pixel values
def denormalize_coords(crops):
    crops.xmin = crops.xmin * crops.im_width
    crops.ymin = crops.ymin * crops.im_height
    crops.xmax = crops.xmax * crops.im_width
    crops.ymax = crops.ymax * crops.im_height
    # Round results to 2 decimal places
    crops.round(2)
    #print("De-normalized cropping coordinates: \n", crops.head())

    return crops

# For images with >1 detection, make a 'super box' that containings all boxes
def make_superboxes(crops):
    # Get superbox coordinates that contain all detection boxes per image
    xmin = pd.DataFrame(crops.groupby(['url'])['xmin'].min()) # smallest xmin
    ymin = pd.DataFrame(crops.groupby(['url'])['ymin'].min()) # smallest ymin
    xmax = pd.DataFrame(crops.groupby(['url'])['xmax'].max()) # largest xmax
    ymax = pd.DataFrame(crops.groupby(['url'])['ymax'].max()) # largest ymax

    # Workaround to get im_height, im_width and class in same format as 'super box' coords
    # There is only one value for im_height and im_width, so taking max will return unchanged values
    im_h = pd.DataFrame(crops.groupby(['url'])['im_height'].max())
    im_w = pd.DataFrame(crops.groupby(['url'])['im_width'].max())
    im_class = pd.DataFrame(crops.groupby(['url'])['class_id'].max())
  
    # Make list of superboxes
    superbox_list = [im_h, im_w, xmin, ymin, xmax, ymax, im_class]

    # Make a new dataframe with 1 superbox per image
    superbox_df = reduce(lambda  left, right: pd.merge(left, right, on=['url'],
                                            how='outer'), superbox_list)
    #print("Cropping dataframe, 1 superbox per image: \n", crops_unq.head())

    return superbox_df

# Add EOL img identifying info from breakdown file to cropping data
def add_identifiers(*, bundle_info, crops):
    # Get dataObjectVersionIDs, identifiers, and eolMediaURLS from indexed cols
    ids = bundle_info.iloc[:, np.r_[0:2,-2]]
    ids.set_index('eolMediaURL', inplace=True, drop=True)
    #print("Bundle identifying info head: \n", ids.head())

    # Set up superboxes df for mapping to bundle_info
    superboxes.reset_index(inplace=True)
    superboxes.rename(columns={'url': 'eolMediaURL'}, inplace=True)
    superboxes.set_index('eolMediaURL', inplace=True, drop=True)

    # Map dataObjectVersionIDs to crops_unq using eolMediaURL as the index
    crops_w_identifiers = pd.DataFrame(superboxes.merge(ids, left_index=True, right_index=True))
    crops_w_identifiers.reset_index(inplace=True)
    print("\n Crops with added EOL identifiers: \n", crops_w_identifiers.head())
  
    return crops_w_identifiers

In [ ]:
# For images with >1 detection, make a 'super box' that containings all boxes

# Read in crop file exported from "Combine output files A-D" block above
concat_outfpath = "/content/drive/My Drive/train/results/aves_cropcoords_tf2_ssd_concat.tsv" #@param {type:"string"}
crops = read_datafile(concat_outfpath, sep='\t', header=0, disp_head=False)

# De-normalize cropping coordinates to pixel values
crops = denormalize_coords(crops)

# Make 1 superbox per image [coordinates: bottom left (smallest xmin, ymin) and top right (largest xmax, ymax)]
superboxes = make_superboxes(crops)

# Read in EOL image "breakdown" bundle dataframe from "breakdown_download" bundle used for cropping
bundle = "https://editors.eol.org/other_files/bundle_images/files/images_for_Aves_20K_breakdown_download_000001.txt" #@param {type:"string"}
breakdown = bundle.replace("download_", "") # Get EOL breakdown bundle url from "breakdown_download" address
bundle_info = read_datafile(breakdown, sep='\t', header=0, disp_head=False)

# Add EOL img identifying info from breakdown file to cropping data
crops_w_identifiers = add_identifiers(bundle_info=bundle_info, crops=superboxes)

#### Make superbox dimensions square (Optional: Add padding)

In [ ]:
# Define functions

# Suppress pandas warning about writing over a copy of data
pd.options.mode.chained_assignment = None  # default='warn'

# Check if dimensions are out of bounds
def are_dims_oob(dim):
    # Check if square dimensions are out of image bounds (OOB)
    if dim > min(im_h, im_w):
        return True
    else:
        return False

# Center padded, square coordinates around object midpoint
def center_coords(coord_a, coord_b, crop_w, crop_h, im_dim_a, im_dim_b, pad):
    # Centered, padded top-right coordinates
    tr_coord_a = coord_a + 0.5*(abs(crop_h - crop_w)) + pad
    tr_coord_b = coord_a + pad
    # Adjust coordinate positions if OOB (out of bounds)
    if crop_h != crop_w: # for cond 1 and 2
        # Both coords not OOB
        if (tr_coord_a <= im_dim_a) and (tr_coord_b <= im_dim_b):
            bl_coord_a = coord_a - 0.5*(abs(crop_h - crop_w)) - pad
            bl_coord_b = coord_b - pad
        # Topright coord_a OOB (+), shift cropping box down/left a-axis 
        elif (tr_coord_a > im_dim_a) and (tr_coord_b <= im_dim_b):
            bl_coord_a = 0.5*(abs(im_dim_a - crop_w))
            bl_coord_b = coord_b - pad
        # Topright coord_b OOB (+), shift cropping box down/left b-axis    
        elif (tr_coord_a <= im_dim_a) and (tr_coord_b > im_dim_b):
            bl_coord_a = coord_a - 0.5*(abs(crop_h - crop_w)) - pad
            bl_coord_b = coord_b - (tr_coord_b - im_dim_b + pad)
        # Both coords OOB (+), shift cropping box down/left both axes     
        elif (tr_coord_a > im_dim_a) and (tr_coord_b > im_dim_b):
            bl_coord_a = 0.5*(abs(im_dim_a - crop_w))
            bl_coord_b = coord_b - (tr_coord_b - im_dim_b + pad)
    else: # for cond 3
        # Both coords not OOB
        if (tr_coord_a <= im_dim_a) and (tr_coord_b <= im_dim_b):
            bl_coord_a = coord_a - pad
            bl_coord_b = coord_b - pad
        # Topright coord_a OOB (+), shift cropping box down/left a-axis 
        elif (tr_coord_a > im_dim_a) and (tr_coord_b <= im_dim_b):
            bl_coord_a = coord_a - (tr_coord_a - im_dim_a + pad)
            bl_coord_b = coord_b - pad
        # Topright coord_b OOB (+), shift cropping box down/left b-axis    
        elif (tr_coord_a <= im_dim_a) and (tr_coord_b > im_dim_b):
            bl_coord_a = coord_a - pad
            bl_coord_b = coord_b - (tr_coord_b - im_dim_b + pad)
        # Both coords OOB (+), shift cropping box down/left both axes     
        elif (tr_coord_a > im_dim_a) and (tr_coord_b > im_dim_b):
            bl_coord_a = coord_a - (tr_coord_a - im_dim_a + pad)
            bl_coord_b = coord_b - (tr_coord_b - im_dim_b + pad)
    
    return bl_coord_a, bl_coord_b

# Set square dimensions = larger bounding box side
def make_large_square(dim):
    # Set new square crop dims = original larger crop dim
    lg_square = crop_w1 = crop_h1 = dim
    return lg_square

# Set square dimensions = smaller bounding box side
def make_small_square(dim):
    # Set new square crop dims = original smaller crop dim
    sm_square = crop_w1 = crop_h1 = dim
    return sm_square

# Add x% padding to bounding box dimensions
def add_padding(dim):
    # Add padding on all sides of square
    padded_dim = dim + 2*percent_pad*dim
    return padded_dim

# Make square crops that are within image bounds for different scenarios
def make_square_crops(df):
    print("Before making square: \n", df.head())
    start_time = time.time()
    df['crop_height'] = round(df['ymax'] - df['ymin'], 1)
    df['crop_width'] = round(df['xmax'] - df['xmin'], 1)
    for i, row in df.iterrows():
        # Define variables for use filtering data through loops below
        crop_h0 = df['crop_height'][i]
        crop_w0 = df['crop_width'][i]
        #print("crop_h0: {}, crop_w0: {}".format(crop_h0, crop_w0))
        pad = percent_pad * max(crop_h0, crop_w0)  
        global im_h, im_w
        im_h = df.im_height[i]
        im_w = df.im_width[i]
        xmin0 = df.xmin[i]
        ymin0 = df.ymin[i]
        xmax0 = df.xmax[i]
        ymax0 = df.ymax[i]
        
        # Conditions determine how rectangle bounding boxes are made square
        cond1 = crop_h0 > crop_w0 # crop height > width
        cond2 = crop_h0 < crop_w0 # crop width > height
        cond3 = crop_h0 == crop_w0 # crop height = width (already square)

        # Crop Height > Crop Width
        # See project wiki "Detailed explanation with drawings: convert_bboxdims.py", Scenario 1
        if cond1:
            lg_sq = make_large_square(crop_h0)
            lg_padded_sq = add_padding(lg_sq)
            sm_sq = make_small_square(crop_w0)
            sm_padded_sq = add_padding(sm_sq)

            # Where padded crop height is within image dimensions
            if are_dims_oob(lg_padded_sq) is False:
                # Make new crop dims equal to large padded square dims
                df.crop_width[i] = df.crop_height[i] = crop_h1 = lg_padded_sq  
                # Center position of new crop dims (adjust xmin, ymin)
                df.xmin[i], df.ymin[i] = center_coords(xmin0, ymin0, crop_w0, crop_h1, im_w, im_h, pad)

            # Where unpadded crop height is within image dimensions
            elif (are_dims_oob(lg_padded_sq) is False) and (are_dims_oob(lg_sq) is True):
                # Make new crop dims equal to large padded square dims
                df.crop_width[i] = df.crop_height[i] = crop_h1 = lg_sq  
                # Center position of new crop dims (adjust xmin, ymin)
                df.xmin[i] = xmin0 - 0.5*(min(im_h, im_w) - crop_w0)
                df.ymin[i] = 0

            # Where padded crop width is within image dimensions
            elif (are_dims_oob(lg_sq) is False) and (are_dims_oob(sm_padded_sq) is True):
                # Make new crop dimensions equal to small padded square dims
                df.crop_width[i] = df.crop_height[i] = crop_w1 = sm_padded_sq
                # Center position of new crop dims (adjust xmin, ymin)
                df.xmin[i] = xmin0 - 0.5*pad
                df.ymin[i] = ymin0 + 0.5*(crop_h0 - crop_w0) - pad   

            # Where unpadded crop width is within image dimensions
            elif (are_dims_oob(sm_padded_sq) is False) and (are_dims_oob(sm_sq) is True):
                # Make new crop dimensions equal to small padded square dims
                df.crop_width[i] = df.crop_height[i] = crop_w1 = sm_sq

            # Where crop width and height are both OOB
            elif are_dims_oob(sm_sq) is False:
                # Do not crop, set values equal to image dimensions
                df.crop_height[i] = crop_h1 = im_h 
                df.ymin[i] = 0
                df.xmin[i] = 0 
    
        # Crop Width > Crop Height
        # See project wiki "Detailed explanation with drawings: convert_bboxdims.py", Scenario 2
        elif cond2:
            lg_sq = make_large_square(crop_w0)
            lg_padded_sq = add_padding(lg_sq)
            sm_sq = make_small_square(crop_h0)
            sm_padded_sq = add_padding(sm_sq)

            # Where padded crop width is within image dimensions
            if are_dims_oob(lg_padded_sq) is False:
                # Make new crop dims equal to large padded square dims
                df.crop_width[i] = df.crop_height[i] = crop_w1 = lg_padded_sq  
                # Center position of new crop dims (adjust xmin, ymin)
                df.ymin[i], df.xmin[i] = center_coords(ymin0, xmin0, crop_w1, crop_h0, im_w, im_h, pad)

            # Where unpadded crop width is within image dimensions
            elif (are_dims_oob(lg_padded_sq) is False) and (are_dims_oob(lg_sq) is True):
                # Make new crop dims equal to large padded square dims
                df.crop_width[i] = df.crop_height[i] = crop_w1 = lg_sq  
                # Center position of new crop dims (adjust xmin, ymin)
                df.ymin[i] = ymin0 - 0.5*(min(im_h, im_w) - crop_h0)
                df.xmin[i] = 0

            # Where padded crop height is within image dimensions
            elif (are_dims_oob(lg_sq) is False) and (are_dims_oob(sm_padded_sq) is True):
                # Make new crop dimensions equal to small padded square dims
                df.crop_width[i] = df.crop_height[i] = crop_h1 = sm_padded_sq
                # Center position of new crop dims (adjust xmin, ymin)
                df.ymin[i] = ymin0 - pad
                df.xmin[i] = xmin0 + 0.5*(crop_w0 - crop_h0) - pad   

            # Where unpadded crop height is within image dimensions
            elif (are_dims_oob(sm_padded_sq) is False) and (are_dims_oob(sm_sq) is True):
                # Make new crop dimensions equal to small padded square dims
                df.crop_width[i] = df.crop_height[i] = crop_h1 = sm_sq

            # Where crop width and height are both OOB
            elif are_dims_oob(sm_sq) is False:
                # Do not crop, set values equal to image dimensions
                df.crop_width[i] = crop_w1 = im_w
                df.crop_height[i] = crop_h1 = im_h 
                df.ymin[i] = 0
                df.xmin[i] = 0 

        # Crop Width == Crop Height
        # See project wiki "Detailed explanation with drawings: convert_bboxdims.py", Scenario 3
        elif cond3: 
            lg_sq = make_large_square(crop_w0)
            lg_padded_sq = add_padding(lg_sq)
            sm_sq = make_small_square(crop_h0)
            sm_padded_sq = add_padding(sm_sq)
        
            # Where padded crop width/height is within image dimensions
            if are_dims_oob(lg_padded_sq) is False:            
                # Make new crop dims equal to large padded square dims
                df.crop_width[i] = df.crop_height[i] = crop_w1 = crop_h1 = lg_padded_sq
                # Center position of new crop dims (adjust xmin, ymin)
                df.xmin[i], df.ymin[i] = center_coords(xmin0, ymin0, crop_w0, crop_w1, im_w, im_h, pad)
                
            # Where unpadded crop width/height is within image dimensions
            elif (are_dims_oob(lg_padded_sq) is True) and (are_dims_oob(lg_sq) is False):
                # Both coords not OOB, no changes needed
                if (ymax0 <= im_h) and (xmax0 <= im_w):
                    pass
                
                # Topright X coord OOB (+), shift cropping box left
                elif (ymax0 <= im_h) and (xmax0 > im_w):  
                    df.xmin[i] = xmin0 - (xmax0 - im_w)
                # Topright Y coord OOB (+), shift cropping box down
                elif (ymax0 > im_h) and (xmax0 <= im_w):
                    df.ymin[i] = ymin0 - (ymax0 - im_h)
                # X and Y coords OOB (+), shift cropping box down and left   
                elif (ymax0 > im_h) and (xmax0 > im_w):
                    df.ymin[i] = ymin0 - (ymax0 - im_h)
                    df.xmin[i] = xmin0 - (xmax0 - im_w)

    # Image coordinates should be positive, set negative xmin and ymin values to 0
    df.xmin[df.xmin < 0] = 0
    df.ymin[df.ymin < 0] = 0
    print("Cropping coordinates, made square and with {}% padding: \n{}".format(percent_pad, df.head()))

    # Print time to run script
    print ('Run time: {} seconds'.format(format(time.time()- start_time, '.2f')))

    return df

# Format cropping dimensions to EOL standards
def format_crops_for_eol(df):
# {"height":"423","width":"640","crop_x":123.712,"crop_y":53.4249,"crop_width":352,"crop_height":0}
    df['crop_dimensions'] = np.nan
    for i, row in df.iterrows():
        df.crop_dimensions[i] = ('{{"height":"{}","width":"{}","crop_x":{},"crop_y":{},"crop_width":{},"crop_height":{}}}'
        .format(df.im_height[i], df.im_width[i], df.xmin[i], df.ymin[i], df.crop_width[i], df.crop_height[i]))
    #print("\n EOL formatted cropping dimensions: \n", df.head())

    # Add other dataframe elements from cols: identifier, dataobjectversionid, eolmediaurl, im_class, crop_dimensions
    eol_crops = pd.DataFrame(df.iloc[:,np.r_[-5,-4,-6,0,-1]])
    print("\n EOL formatted cropping dimensions: \n", eol_crops.head())

    return eol_crops

In [ ]:
# Make crops square and within image bounds

# Optional TO DO: Pad by xx% larger crop dimension
percent_pad = 0 #@param {type:"slider", min:0, max:10, step:2}

# Make crops square and within bounds
df = make_square_crops(crops_w_identifiers)

# Export crop coordinates to display_test.tsv to visualize results in next code block and confirm crop transformations
display_test_fpath = os.path.splitext(concat_outfpath)[0] + '_displaytest' + '.tsv'
print("\n File for displaying square crops on images will be saved to: \n", display_test_fpath)
df.to_csv(display_test_fpath, sep='\t', index=False)

# Format image and cropping dimensions for EOL standards
eol_crops = format_crops_for_eol(df)

# Write results to tsv
eol_crops_fpath = os.path.splitext(display_test_fpath)[0].rsplit('_',2)[0] + '_20k_final' + '.tsv'
eol_crops.to_csv(eol_crops_fpath, columns = eol_crops.iloc[:,:-1], sep='\t', index=False)

## Display cropping results on images
---

In [ ]:
# Define functions

import cv2

# Read in cropping file for displaying results
# Note: If you just ran "Post-process results" above, you do not need to enter anything
# TO DO: If you ran "Generate crops" during a previous session, enter the path for desired cropping file
if 'outfpath' not in locals() or globals():
    outfpath = "/content/drive/My Drive/train/results/aves_cropcoords_tf2_ssd_concat_displaytest.tsv" #@param {type:"string"}
df = pd.read_csv(outfpath, sep="\t", header=0)
print(df.head())

# For uploading an image from url
# Modified from https://www.pyimagesearch.com/2015/03/02/convert-url-to-image-with-python-and-opencv/
def url_to_image(url):
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    im_h, im_w = image.shape[:2]
 
    return image

# Draw cropping box on image
def draw_box_on_image(df, img):
    # Get box coordinates
    xmin = df['xmin'][i].astype(int)
    ymin = df['ymin'][i].astype(int)
    xmax = df['xmin'][i].astype(int) + df['crop_width'][i].astype(int)
    ymax = df['ymin'][i].astype(int) + df['crop_height'][i].astype(int)
    boxcoords = [xmin, ymin, xmax, ymax]

    # Set box/font color and size
    maxdim = max(df['im_height'][i],df['im_width'][i])
    fontScale = maxdim/600
    box_col = (255, 0, 157)
  
    # Add label to image
    tag = df['class_id'][i]
    image_wbox = cv2.putText(img, tag, (xmin+7, ymax-12), cv2.FONT_HERSHEY_SIMPLEX, fontScale, box_col, 2, cv2.LINE_AA)  
  
    # Draw box label on image
    image_wbox = cv2.rectangle(img, (xmin, ymax), (xmax, ymin), box_col, 5)

    return image_wbox, boxcoords

In [ ]:
# Display crop dimensions on images

# TO DO: Adjust line below to see up to 50 images displayed at a time
start = 0 #@param {type:"slider", min:0, max:5000, step:50}
stop = start+50

# Loop through images
for i, row in df.iloc[start:stop].iterrows():
  # Read in image 
  url = df['eolMediaURL'][i]
  img = url_to_image(url)
  
  # Draw bounding box on image
  image_wbox, boxcoords = draw_box_on_image(df, img)
  
  # Plot cropping box on image
  _, ax = plt.subplots(figsize=(10, 10))
  ax.imshow(image_wbox)

  # Display image URL and coordinatesabove image
  # Helps with fine-tuning data transforms in post-processing steps above
  plt.title('{} \n xmin: {}, ymin: {}, xmax: {}, ymax: {}'.format(url, boxcoords[0], boxcoords[1], boxcoords[2], boxcoords[3]))